<a href="https://colab.research.google.com/github/SeokcheonMoon/study_data_analytics/blob/main/docs/quests/CDAs/CDA_categorical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime 이후 상단바 런타임의 Restart Session 을 클릭해야함
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [2]:
df_fruitstore = pd.read_csv("/content/drive/MyDrive/fruitstore_saleshistory.csv")

In [3]:
df_fruitstore

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05
3,2018-01-02,화,1,1,1023108864,정상회원,반찬,두부/유부,두부,연두부(100g),1.0,수지구,풍덕천동,여,36.0,30대이하,766,매장C,NaN,10:08
4,2018-01-02,화,1,1,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,1.0,수지구,풍덕천동,여,34.0,30대이하,4403,매장C,NaN,10:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705566,2018-06-30,토,6,26,146297344,정상회원,수산,건어물,멸치류,국물멸치(300g),1.0,수지구,성복동,여,51.0,50대,9189,매장A,NaN,17:56
705567,2018-06-30,토,6,26,1532985472,정상회원,수산,해조,미역/다시마,자른다시마(100g),1.0,수지구,동천동,여,63.0,60대,5169,매장A,NaN,14:37
705568,2018-06-30,토,6,26,1532985472,정상회원,반찬,두부/유부,두부,두부(1모:420g),2.0,수지구,동천동,여,63.0,60대,7466,매장A,NaN,14:37
705569,2018-06-30,토,6,26,1209779776,정상회원,간식,과자,스낵,하늘바다새우(60g),1.0,수지구,동천동,여,41.0,40대,3542,매장A,NaN,18:04


## 가설 1-2 (분석내용): 요일(X)이 가장 인기 있는 과일 카테고리 판매(Y)에 영향을 미칩니까?

- 귀무가설 : 요일은 가장 인기있는 과일 카테고리 판매에 영향을 미치지 않는다.(관련성이 없다.) (p>0.05)
- 대립가설 : 요일은 가장 인기있는 과일 카테고리 판매에 영향을 미친다.(관련성이 있다.)  (p<0.05)
- 분석 columns : 요일, 구매수량

In [56]:
df_fruitstore["물품중분류"].value_counts()

잎/줄기채소    51436
두부/유부     50502
알         41429
과자        35429
양념채소      28164
          ...  
식기/편백       714
비물품         569
과실주         160
섬유          120
도서           13
Name: 물품중분류, Length: 62, dtype: int64

In [84]:
condition_과일 = "물품중분류=='과일'"
condition_잎_줄기채소 = "물품중분류=='잎/줄기채소'"
condition_두부_유부 = "물품중분류=='두부/유부'"
condition_알 = "물품중분류=='알'"
condition_과자 = "물품중분류=='과자'"
condition_양념채소 = "물품중분류=='양념채소'"
conditions = f"{condition_과일} or {condition_잎_줄기채소} or {condition_두부_유부} or {condition_알} or {condition_과자} or {condition_양념채소}"
df_fruitstore_conditions = df_fruitstore.query(conditions)
df_fruitstore_conditions

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05
3,2018-01-02,화,1,1,1023108864,정상회원,반찬,두부/유부,두부,연두부(100g),1.0,수지구,풍덕천동,여,36.0,30대이하,766,매장C,NaN,10:08
5,2018-01-02,화,1,1,716128320,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,51.0,50대,7083,매장C,NaN,10:10
7,2018-01-02,화,1,1,743192512,정상회원,채소,잎/줄기채소,어린잎채소,어린잎채소(100g),2.0,수지구,성복동,여,51.0,50대,11104,매장C,NaN,10:12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705555,2018-06-30,토,6,26,886321664,정상회원,반찬,두부/유부,두부,두부(1모:420g),1.0,기타,기타,여,42.0,40대,3733,매장A,NaN,12:36
705556,2018-06-30,토,6,26,572193664,정상회원,반찬,두부/유부,콩가공/기타,콩국물(300g),1.0,수지구,풍덕천동,여,46.0,40대,2202,매장A,NaN,13:48
705561,2018-06-30,토,6,26,1333795840,정상회원,채소,잎/줄기채소,브로컬리,브로컬리(250g),1.0,수지구,풍덕천동,여,41.0,40대,3982,매장A,NaN,15:28
705568,2018-06-30,토,6,26,1532985472,정상회원,반찬,두부/유부,두부,두부(1모:420g),2.0,수지구,동천동,여,63.0,60대,7466,매장A,NaN,14:37


In [85]:
df_fruitstore_conditions_category_pivot = pd.pivot_table(df_fruitstore_conditions[["물품중분류", "요일"]], index = "물품중분류", columns = ["요일"], aggfunc = len)
df_fruitstore_conditions_category_pivot

요일,금,목,수,월,일,토,화
물품중분류,,,,,,,
과일,2552,2321,2213,2586,1169,2347,2214
과자,5577,5227,5122,5925,3398,4931,5249
두부/유부,7998,7234,7549,9555,3632,6392,8142
알,6391,6086,6258,7576,2964,5362,6792
양념채소,4376,4112,4205,4968,2126,3994,4383
잎/줄기채소,8277,7919,7599,8973,2927,7807,7934


In [86]:
stats.chi2_contingency(observed = df_fruitstore_conditions_category_pivot, correction = True )

Chi2ContingencyResult(statistic=785.7044709153745, pvalue=6.038930520662868e-146, dof=30, expected_freq=array([[2436.13451039, 2278.76344879, 2282.0189241 , 2741.73359657,
        1123.20824601, 2135.66106619, 2404.48020795],
       [5603.80532195, 5241.80692295, 5249.29544616, 6306.77052284,
        2583.6998408 , 4912.63056188, 5530.99138342],
       [7987.90189871, 7471.89401966, 7482.56847843, 8989.93832579,
        3682.91539022, 7002.67206627, 7884.10982092],
       [6552.82538833, 6129.52155044, 6138.27827596, 7374.83970732,
        3021.25661759, 5744.5982542 , 6467.68020615],
       [4454.70019158, 4166.93246148, 4172.88540308, 5013.51675196,
        2053.89151024, 3905.25634776, 4396.8173339 ],
       [8135.63268904, 7610.08159668, 7620.95347227, 9156.20109551,
        3751.02839514, 7132.18170371, 8029.92104766]]))

In [88]:
round(6.038930520662868e-146,6)

0.0

### 분석결과
- 통계결과
  + p-value = 6.038930520662868e-146,6
  + p-value <0.05 이므로 대립가설이 True
- 사실 결론 : 요일은 가장 인기있는 과일 카테고리 판매에 영향을 미친다. (관련성이 있다.)